In [1]:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F

    class BinaryClassificationCNN(nn.Module):
        def __init__(self):
            super(BinaryClassificationCNN, self).__init__()
            
            # 첫 번째 Convolutional 레이어: 3개의 입력 채널(RGB), 32개의 출력 채널, 2*2 커널
            self.conv1 = nn.Conv2d(3, 32, kernel_size=2, padding=1)
            # 두 번째 Convolutional 레이어
            self.conv2 = nn.Conv2d(32, 64, kernel_size=2, padding=1)
            # 세 번째 Convolutional 레이어
            self.conv3 = nn.Conv2d(64, 128, kernel_size=2, padding=1)
            
            # Pooling 레이어
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
            
            # Dropout 레이어
            self.dropout = nn.Dropout(0.5)
            
            # 첫 번째 Fully Connected 레이어
            self.fc1 = nn.Linear(128 * 4 * 4, 128)
            
            # 두 번째 Fully Connected 레이어
            self.fc2 = nn.Linear(128, 1)

        def forward(self, x):
            # Convolutional 레이어와 활성화 함수 적용
            x = F.relu(self.conv1(x))
            x = self.pool(x)
            x = F.relu(self.conv2(x))
            x = self.pool(x)
            x = F.relu(self.conv3(x))
            x = self.pool(x)
            # Flatten
            x = x.view(-1, 128 * 4 *4)
            # Dropout 적용
            x = self.dropout(x)
            # Fully Connected 레이어와 활성화 함수 적용
            x = F.relu(self.fc1(x))
            x = self.dropout(x)
            # 출력 레이어
            x = torch.sigmoid(self.fc2(x))
            return x

    # 모델 인스턴스화
    model = BinaryClassificationCNN()

In [2]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CustomDataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, dir_path_ball, dir_path_background, transform=None):
        self.dir_path_ball = dir_path_ball
        self.dir_path_background = dir_path_background
        self.transform = transform
        
        # 이미지 파일 경로와 라벨을 리스트로 만듭니다.
        self.image_paths = []
        self.labels = []

        # 라벨이 1인 이미지 (테니스공)
        for filename in os.listdir(self.dir_path_ball):
            if filename.endswith('.png'):  # .png 파일만 추가
                self.image_paths.append(os.path.join(self.dir_path_ball, filename))
                self.labels.append(1)
        
        # 라벨이 0인 이미지 (배경)
        for filename in os.listdir(self.dir_path_background):
            if filename.endswith('.png'):  # .png 파일만 추가
                self.image_paths.append(os.path.join(self.dir_path_background, filename))
                self.labels.append(0)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        # 이미지를 로드하고 라벨을 가져옵니다.
        image = Image.open(self.image_paths[index])
        label = self.labels[index]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 데이터셋 인스턴스화 및 DataLoader 설정
transform = transforms.Compose([
    transforms.Resize((30, 30)), # 이미지 크기 조정
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dir_path_ball = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30\cropped_ball\augmentation'
dir_path_background = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30\cropped_baseground'

train_dataset = CustomDataset(
    dir_path_ball=dir_path_ball,
    dir_path_background=dir_path_background,
    transform=transform
)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)

# 모델 인스턴스화 및 학습 준비
model = BinaryClassificationCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 과정
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # 데이터와 타겟을 GPU로 옮김 (GPU 사용 시)
        data = data.to(device)
        targets = targets.to(device).float().view(-1, 1)

        # 순전파
        scores = model(data)
        loss = criterion(scores, targets.float())

        # 역전파
        optimizer.zero_grad()
        loss.backward()

        # 가중치 갱신
        optimizer.step()

        # 학습 진행 상황 출력
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

print("Training complete.")


Epoch [1/10], Step [0/1044], Loss: 0.9720
Epoch [1/10], Step [100/1044], Loss: 0.6931
Epoch [1/10], Step [200/1044], Loss: 0.6931
Epoch [1/10], Step [300/1044], Loss: 0.6931
Epoch [1/10], Step [400/1044], Loss: 0.6931
Epoch [1/10], Step [500/1044], Loss: 0.6931
Epoch [1/10], Step [600/1044], Loss: 0.6931
Epoch [1/10], Step [700/1044], Loss: 0.6931
Epoch [1/10], Step [800/1044], Loss: 0.6931
Epoch [1/10], Step [900/1044], Loss: 0.6931
Epoch [1/10], Step [1000/1044], Loss: 0.6931
Epoch [2/10], Step [0/1044], Loss: 0.6931
Epoch [2/10], Step [100/1044], Loss: 0.6931
Epoch [2/10], Step [200/1044], Loss: 0.6931
Epoch [2/10], Step [300/1044], Loss: 0.6931
Epoch [2/10], Step [400/1044], Loss: 0.6931
Epoch [2/10], Step [500/1044], Loss: 0.6931
Epoch [2/10], Step [600/1044], Loss: 0.6931
Epoch [2/10], Step [700/1044], Loss: 0.6931
Epoch [2/10], Step [800/1044], Loss: 0.6931
Epoch [2/10], Step [900/1044], Loss: 0.6931
Epoch [2/10], Step [1000/1044], Loss: 0.6931
Epoch [3/10], Step [0/1044], Loss:

In [3]:
torch.save(model, 'model_v2.pt')